# Train Cogames

this notebook will help you to get started with cogames with the simplest training example.

Make sure you have selected T4 runtime!

## Install Dependencies

In [ ]:
!git clone https://github.com/Metta-AI/metta.git --depth 1
%cd metta

fatal: destination path 'metta' already exists and is not an empty directory.
/content/metta/metta


### Install Bazel

In [ ]:
!sudo apt install apt-transport-https curl gnupg -y
!curl -fsSL https://bazel.build/bazel-release.pub.gpg | sudo apt-key add -
!echo "deb [arch=amd64] https://storage.googleapis.com/bazel-apt stable jdk1.8" | sudo tee /etc/apt/sources.list.d/bazel.list
!sudo apt update && sudo apt install bazel -y
!bazel --version


### Install Nim

In [ ]:
!sudo apt-get update -y
!sudo apt-get install -y curl git build-essential
!curl https://nim-lang.org/choosenim/init.sh -sSf | sh -s -- -y
import os
os.environ["PATH"] += ":/root/.nimble/bin"


!nim --version
!nimble --version


### Install all the required packages

In [ ]:
!uv run cogames

## Training Code + PPO Loss

It includes a complete reinforcement learning training loop that implements a rollout buffer for generalized advantage estimation (GAE), a PPO update function with clipping, entropy regularization, and value loss computation, and supports both simple and LSTM-based policies. It interacts with the MettaGridEnv simulation, collecting observations, actions, and rewards over multiple epochs to optimize the policy network.

You can modify mission configurations, hyperparameters, and policy types to experiment with different training setups.

In [ ]:
!uv run cogames train --mission training_facility.harvest --variant lonely_heart --variant heart_chorus --variant pack_rat --variant extractor_base

## View Replays in Mettascope

After training, you can view replays directly in this notebook using Mettascope. The replay viewer will load in an iframe and receive replay data via postMessage, allowing you to visualize your training results without needing to download files or set up a local server.


In [ ]:
import base64
from pathlib import Path
from IPython.display import Javascript, display, HTML

# Load dinky7.json.z from mettascope replays folder
replay_path = Path("packages/mettagrid/nim/mettascope/tests/data/replays/dinky7.json.z")

# Read the file as binary
with open(replay_path, 'rb') as f:
    data = f.read()

# Base64 encode
b64 = base64.b64encode(data).decode('utf-8')

# Display iframe to mettascope
iframe_src = "https://metta-ai.github.io/metta/mettascope/mettascope.html"
# local iframe for testing
# iframe_src = "http://localhost:8000/mettascope.html"
iframe_id = "mettascope_iframe"

display(HTML(f'''
<div>
    <p>Loading replay: <code>{replay_path.name}</code></p>
    <iframe id="{iframe_id}" src="{iframe_src}" width="100%" height="800" style="border: 1px solid #ccc;"></iframe>
</div>
'''))

# Send data via postMessage after iframe loads
b64_escaped = b64.replace('\\', '\\\\').replace("'", "\\'").replace('\n', '\\n')

display(Javascript(f'''
(function() {{
    const iframe = document.getElementById('{iframe_id}');
    const fileName = 'dinky7.json.z';
    const base64Data = '{b64_escaped}';

    function sendReplayData() {{
        console.log('Sending replay data to mettascope...');
        iframe.contentWindow.postMessage({{
            type: 'replayData',
            base64: base64Data,
            fileName: fileName
        }}, '*');
        console.log('Replay data sent to mettascope');
    }}
    
    window.addEventListener('message', function(event) {{
        if (event.data.type === 'mettascopeReady') {{
            sendReplayData();
        }}
    }});


}})();
'''))


<IPython.core.display.Javascript object>